<a href="https://colab.research.google.com/github/choiwhmarco/COGS_181/blob/main/LeNetTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LeNet

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
#loda data
import glob
train_images_dir ='./data/tiny-imagenet-200/train/'
test_images_dir ='./data/tiny-imagenet-200/test/'
val_images_dir ='./data/tiny-imagenet-200/val/'


In [ ]:
#train label
label_fliepath='./data/tiny-imagenet-200/wnids.txt'
file_object = open(label_fliepath,'r')
label_dict={}
i=0
for line in file_object:
    l=line.rstrip('\n')
    label_dict[l]=i
    i=i+1
file_object.close()

In [ ]:
#train data
traindata=np.zeros([200*500,64,64,3],dtype ='float32')
trainlabel=np.zeros([200*500,200],dtype ='float32')
for key in label_dict:
    path=glob.glob(train_images_dir+key+'/images/*.JPEG')   
    i=0
    for imageFile in path:
        img = cv2.imread(imageFile)  #bgr
        traindata[label_dict[key]*500+i]=img
        trainlabel[label_dict[key]*500+i][label_dict[key]]=1
        i=i+1

In [ ]:
#test data
testdata=np.zeros([10000,64,64,3],dtype ='float32')
path=glob.glob(test_images_dir+'images/*.JPEG')
i=0
for imageFile in path:
    img = cv2.imread(imageFile)  #bgr
    testdata[i]=img
    i=i+1 

In [ ]:
#val label
vallabel=np.zeros([10000,200],dtype ='float32')
valpath=[None]*10000
path=val_images_dir+'/val_annotations.txt'
file_object = open(path,'r')
i=0
for line in file_object:
    l=line.split()
    valpath[i]=l[0]
    vallabel[i][label_dict[l[1]]]=1
    i=i+1
file_object.close()

In [ ]:
#val data
valdata=np.zeros([10000,64,64,3],dtype ='float32')
for i in range(10000):
    img = cv2.imread(val_images_dir+'images/'+valpath[i])  #bgr
    valdata[i]=img

model

In [ ]:
#----Weight Initialization---#
#One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
#Convolution and Pooling
#Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input.
#Our pooling is plain old max pooling over 2x2 blocks
def conv2d(x, W):
     return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):   
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
#----input and output----#
x = tf.placeholder(tf.float32,[ None,64, 64, 3])
y = tf.placeholder(tf.float32,[ None,200])

In [ ]:
#----first convolution layer----#
#64x64x3 ->32x32x32
W_conv1 = weight_variable([3, 3, 3, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
#----second convolution layer----#
#32x32x32->16x16x64
W_conv2 = weight_variable([3,3,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
#----second convolution layer----#
#16x16x64->8x8x64
W_conv3 = weight_variable([3,3,64,64])
b_conv3 = bias_variable([64])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

In [ ]:
#----fully connected layer----#
W_fc1 = weight_variable([16 * 16* 64, 1024])
b_fc1 = bias_variable([1024])
h_pool3_flat = tf.reshape(h_pool3, [-1, 16 * 16* 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

In [ ]:
#----read out layer----#
W_fc2 = weight_variable([1024, 200])
b_fc2 = bias_variable([200])
y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

In [ ]:
cross_entropy = -tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_conv)) 
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

In [ ]:
def next_batch(size=50):
    index=np.random.randint(0,200*500,size)
    x1=traindata[index]
    y1=trainlabel[index]
    return x1,y1

In [ ]:
if 'session' in locals() and tf.session is not None:
    print('Close interactive session')
    tensorflow.session.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [ ]:
for i in range(1000):
    batchx,batchy =next_batch(200)
    train_step.run(session = sess, feed_dict = {x:batchx, y:batchy})
    if (i+1) % 100 == 0:
        train_accuracy = accuracy.eval(session = sess,feed_dict = {x:batchx, y:batchy})
        print("step %d, train_accuracy %g" %(i, train_accuracy))

In [ ]:
train_accuracy = accuracy.eval(session = sess,feed_dict = {x:traindata.reshape(200*500,64,64,3), y_:trainlabel.reshape(200*500,1)})
print("test accuracy %g"%train_accuracy)